In [11]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [12]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

mass_pos = [(i, 0, 0) for i in range(1,11)]

ms = [mss.add(Mass(1, pos)) for pos in mass_pos]
fix = mss.add(Fix((0,0,0)))
for i in range(len(ms)):
    if i==0:
        # mss.add(Spring(1, 50000, (fix, ms[i])))
        mss.add(DistanceConstraint(1, (fix, ms[i])))
    else:
        # mss.add(Spring(1, 50000, (ms[i-1], ms[i])))
        mss.add(DistanceConstraint(1, (ms[i-1], ms[i])))

# mA = mss.add (Mass(1, (1,0,0)))
# mB = mss.add (Mass(2, (2,0,0)))
# f1 = mss.add (Fix( (0,0,0)) )
# mss.add (Spring(1, 200000, (f1, mA)))
# mss.add (Spring(1, 100000, (mA, mB)))

In [ ]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
# for s in mss.springs:
for s in mss.constraints:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

TraitError: positions shape expected to have 3 components, but got (0,) components

In [ ]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [ ]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    # for s in mss.springs:
    for s in mss.constraints:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

ValueError: Newton did not converge